In [ ]:
import os
#from nipype import config
#config.enable_provenance()

import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio

from nipype import Workflow, Node, MapNode, Function, IdentityInterface
from nipype import DataGrabber, DataSink
from nipype.interfaces.fsl import (Merge, FLAMEO, ContrastMgr, GLM,
                                   SmoothEstimate, Cluster, ImageMaths, MultipleRegressDesign,
                                   MultiImageMaths)
import nipype.interfaces.fsl as fsl
import nipype.interfaces.utility as util
from nipype.interfaces.fsl.maths import BinaryMaths

import pandas as pd

In [ ]:
def run_rsfmri_glm(rsfmri_dir, analysis_dir, output_dir):
    wf = Workflow(name='rsfmri')
    wf.base_dir = work_dir
    
    def csv_to_glm():
        import pandas as pd
        ev_file = pd.read_csv(os.path.join(analysis_dir, 'design', 'design.csv'),
                          index_col=0, sep=',|\t| ', engine='python')
        regressors = ev_file.to_dict(orient='list')
            
        contrast_file = pd.read_csv(os.path.join(analysis_dir, 'design', 'contrasts.csv'),
                                index_col=0, sep=',|\t| ', engine='python')
        
        ev_list = contrast_file.columns.tolist()
        
        contrasts = []
        for contrast in contrast_file.index.tolist():
            contrast_vec = contrast_file.loc[contrast].values.tolist()
            con = [contrast, 'T', ev_list, contrast_vec]
            contrasts.append(con)
        num_contrasts = len(contrasts)

        return regressors, contrasts, num_contrasts

    regressors, contrasts, num_contrasts = csv_to_glm()
    
    subj_df = pd.read_csv(os.path.join(analysis_dir, 'sub_list.txt'), header=None, index_col=0)
    sub_list = subj_df.index.tolist()
    num_subs = len(sub_list)

    infosource = Node(IdentityInterface(fields=['subject_id']),
                        name='infosource')

    infosource.iterables = [('subject_id', sub_list)]

    datagrabber = Node(DataGrabber(infields=['subject_id'],
                                    outfields=['rest_ts_smooth', 'parc_summary', 'meants']),           
                       name='datagrabber')

    datagrabber.inputs.base_directory = rsfmri_dir
    datagrabber.inputs.template = '*'

    datagrabber.inputs.field_template = {'rest_ts_smooth': '%s/resting/timeseries/'
                                              'target/rest_01_smooth_trans_masked.nii.gz',
                                         'parc_summary': '%s/resting/parcellations/'
                                              'aparc/rest_01_summary.stats',
                                         'meants': '%s/resting/parcellations/aparc/'
                                              'rest_01_avgwf.txt'}
    datagrabber.inputs.template_args = {'rest_ts_smooth': [['subject_id']],
                                        'parc_summary': [['subject_id']],
                                        'meants': [['subject_id']]}
    datagrabber.inputs.sort_filelist = True

    wf.connect(infosource, 'subject_id', datagrabber, 'subject_id')
    
    roi_list = pd.read_csv(os.path.join(analysis_dir, 'rois.txt'), header=None)
    rois = [tuple(x) for x in roi_list.values]
    
    def get_meants(base_dir, subject_id, roi_fs, meants, summary_file):
        import os
        import pandas as pd

        roi_dir = os.path.join(base_dir, subject_id, 'resting', 'roi_betas', roi_fs[1])
        if not os.path.exists(roi_dir):
            os.makedirs(roi_dir)
        
        summary = pd.read_csv(summary_file, header=None, comment='#', 
                              delim_whitespace=True, index_col=0) 
        roi_idx = summary[summary[4].str.contains(roi_fs[0])].index.values
        meants = pd.read_csv(meants, delim_whitespace=True, usecols=roi_idx-1, 
                             header=None)
        
        ts_filename = os.path.join(roi_dir, 'seed_ts.txt')
        meants.to_csv(ts_filename, header=False, index=False)
        
        beta_filename = os.path.join(roi_dir, 'MNI_beta.nii.gz')
        absbin_filename = os.path.join(roi_dir, 'absbin.nii.gz')

        return roi_fs[1], ts_filename, beta_filename, absbin_filename

    get_meants = pe.Node(Function(input_names=['base_dir', 'subject_id', 'roi_fs', 
                                                'meants', 'summary_file'],
                                  output_names=['roi', 'meants', 'beta_fname', 
                                                'absbin_fname'],
                                  function=get_meants),
                         name='get_meants')
    
    get_meants.inputs.base_dir = rsfmri_dir    
    get_meants.iterables = [('roi_fs', rois)]

    wf.connect(infosource, 'subject_id', get_meants, 'subject_id')
    wf.connect(datagrabber, 'meants', get_meants, 'meants')
    wf.connect(datagrabber, 'parc_summary', get_meants, 'summary_file')
    
    fsl_glm = Node(fsl.GLM(demean=True), name='fsl_glm')

    wf.connect(datagrabber, 'rest_ts_smooth', fsl_glm, 'in_file')
    wf.connect(get_meants, 'meants', fsl_glm, 'design')
    wf.connect(get_meants, 'beta_fname', fsl_glm, 'out_file')

    absbin = Node(fsl.ImageMaths(op_string= '-abs -bin'), 
                  name='absbin')

    wf.connect(fsl_glm, 'out_file', absbin, 'in_file')
    wf.connect(get_meants, 'absbin_fname', absbin, 'out_file')

    base = os.path.join(analysis_dir, 'base.nii.gz')

    mask = pe.JoinNode(interface=fsl.MultiImageMaths(in_file=base),
                joinsource='infosource', joinfield='operand_files', name='mask')

    mask.inputs.op_string = '-add %s ' * (num_subs)

    wf.connect(absbin, 'out_file', mask, 'operand_files')

    merge = pe.JoinNode(interface=fsl.Merge(merged_file='merge.nii.gz', dimension='t'), 
                    joinsource='infosource', joinfield='in_files', name='merge')

    wf.connect(fsl_glm, 'out_file', merge, 'in_files')
        
    thresh = Node(fsl.ImageMaths(op_string='-thr %s -bin' % (num_subs + 1)), 
                  name='thresh')
        
    wf.connect(mask, 'out_file', thresh, 'in_file')
        
    model = Node(fsl.MultipleRegressDesign(), name='model')
    model.inputs.contrasts = contrasts
    model.inputs.regressors = regressors
        
    flameo = Node(fsl.FLAMEO(run_mode='flame1'), name='flameo')
        
    wf.connect(merge, 'merged_file', flameo, 'cope_file')
    wf.connect([(model, flameo, [('design_grp', 'cov_split_file'),
                                 ('design_mat', 'design_file'),
                                 ('design_con', 't_con_file')])])
    
    wf.connect(thresh, 'out_file', flameo, 'mask_file')
        
    smoothest = MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], 
                        name='smoothest')
        
    wf.connect(flameo, 'zstats', smoothest, 'zstat_file')
    wf.connect(thresh, 'out_file', smoothest, 'mask_file')
    
    cluster = MapNode(fsl.Cluster(out_index_file='zstat_index.nii.gz', 
                                  out_localmax_txt_file='zstat_localmax.txt',
                                  out_localmax_vol_file='zstat_localmax.nii.gz',
                                  out_threshold_file='zstat_thresh.nii.gz'), 
                      iterfield=['in_file', 'volume', 'dlh'], name='cluster')
    
    wf.connect(flameo, 'zstats', cluster, 'in_file')
    wf.connect(smoothest, 'volume', cluster, 'volume')
    wf.connect(smoothest, 'dlh', cluster, 'dlh')
    
    cluster.inputs.threshold = 2.3
    cluster.inputs.pthreshold = 0.001

        
    def get_subs(rois, num_contrasts):
        subs = [('base_maths_maths', 'mask_thresh')]

        for roi in rois:
            subs.append(('_roi_fs_%s.%s/' % (roi[0], roi[1]), ''))
        
        for i in range(num_contrasts):
            subs.append(('_cluster%d/zstat' % i, 'zstat%d' % (i + 1)))

        return subs

            
    subsgen = pe.Node(Function(input_names=['rois', 'num_contrasts'],
                                   output_names=['substitutions'],
                                   function=get_subs),
                      name='subsgen')
    
    subsgen.inputs.rois = rois
    subsgen.inputs.num_contrasts = num_contrasts
    
    datasink = pe.Node(interface=nio.DataSink(),
                           name="datasink")
    datasink.inputs.base_directory = output_dir

    wf.connect(get_meants, 'roi', datasink, 'container')
    wf.connect(subsgen, 'substitutions', datasink, 'substitutions')
    
    wf.connect(merge, 'merged_file', datasink, 'qa.merged_betas')

    wf.connect(thresh, 'out_file', datasink, 'qa.mask')
    
    wf.connect([(model, datasink,
                        [('design_grp', 'qa.model'),
                        ('design_mat', 'qa.model.@mat'),
                        ('design_con', 'qa.model.@con')])])
                
    wf.connect([(flameo, datasink,
                        [('copes', 'flameo'),
                        ('pes', 'flameo.@pes'),
                        ('var_copes', 'flameo.@varcopes'),
                        ('zstats', 'flameo.@zstats'),
                        ('tdof', 'flameo.@tdof'),
                        ('tstats', 'flameo.@tstats')])])

    wf.connect([(cluster, datasink,
                        [('index_file', 'cluster.@index'),
                        ('localmax_txt_file', 'cluster.@lmax_txt'),
                        ('localmax_vol_file', 'cluster.@lmax_vol'),
                        ('threshold_file', 'cluster.@thresh')])])
    
    return wf

  
analysis_dir = '/om/user/annepark/git_repos/EF4/group_analysis/'
rsfmri_dir = '/om/user/annepark/git_repos/EF4/'
output_dir = '/om/user/annepark/git_repos/EF4/group_analysis/output_dir'

work_dir = os.path.join(rsfmri_dir, 'group_analysis_wd')
wf = run_rsfmri_glm(rsfmri_dir, analysis_dir, output_dir) 
wf.write_graph()
wf.run(plugin='SLURM')
